In [3]:
from imblearn.datasets import fetch_datasets
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score,accuracy_score,roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans
import random

In [4]:
fetch_dataset=fetch_datasets()
data_name=[]
for word in fetch_dataset:
    data_name.append(word)
import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(random_state=6)
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
from sklearn.neural_network import MLPClassifier
MLP=MLPClassifier()
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()

In [ ]:
finish=dict()

In [ ]:

test_split=[0.2,0.4,0.9]
minor_result=[]
major_result=[]
both_result=[]
for word in data_name:
    if word in finish:
        pass
    else:
        print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        #print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        original=[]
        for n in range(d):
            original.append(n)
        changelist=[]
        for n in range(1,d):
            change=original[n:d]+original[0:n]
            changelist.append(change)
        minor_table=[]
        major_table=[]
        both_table=[]
        for dim in range(1,10):
            if dim%10==0:
                print(dim)
            minor_line=[]
            major_line=[]
            both_line=[]
            for split in test_split:
                if d <= dim:
                    break
                minor_score=[]
                major_score=[]
                result=[]
                for random_value in range(5):
                    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                    positive=X_train[y_train==1]
                    negative=X_train[y_train==-1]
                    Self_train=positive[:]
                    if d <= dim:
                        Self_label=np.zeros(len(positive)*(d))
                    else:
                        Self_label=np.zeros(len(positive)*(dim+1))

                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            increased=positive[:,order]
                            Self_train=np.concatenate([Self_train,increased])
                            Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                    DT.fit(Self_train,Self_label)
                    value1=DT.predict_proba(X_test).T[0]
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            Self_test = X_test[:,order]
                            value1+=DT.predict_proba(Self_test).T[n+1]

                    minor_score.append(roc_auc_score(y_test,value1))
                    Self_train=negative[:]
                    if d <= dim:
                        Self_label=np.zeros(len(negative)*(d))
                    else:
                        Self_label=np.zeros(len(negative)*(dim+1))
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            order=changelist[n]
                            #print(order)
                            increased=negative[:,order]
                            Self_train=np.concatenate([Self_train,increased])
                            Self_label[len(negative)*(n+1):len(negative)*(n+2)]=n+1
                    DT.fit(Self_train,Self_label)

                    value2=DT.predict_proba(X_test).T[0]
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            Self_test = X_test[:,order]
                            value2+=DT.predict_proba(Self_test).T[n+1]
                    major_score.append(roc_auc_score(y_test,-value2))
                    #print(len(LR.predict_log_proba(X_test).T))
                    final_score=value1-value2
                    result.append(roc_auc_score(y_test,final_score))
                minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
                major_line.append(np.array([np.array(major_score).mean(),np.array(major_score).std()]))
                both_line.append(np.array([np.array(result).mean(),np.array(result).std()]))
            minor_table.append(minor_line)
            major_table.append(major_line)
            both_table.append(both_line)
        minor_result.append(minor_table)
        major_result.append(major_table)
        both_result.append(both_table)
        finish[word]=1
makefile(minor_result,"DTmin_5AUC.pkl")
makefile(major_result,"DTmaj_5AUC.pkl")
makefile(both_result,"DTensemble_5AUC.pkl")

In [ ]:
finish=dict()

In [ ]:

test_split=[0.2,0.4,0.9]
minor_result=[]
major_result=[]
both_result=[]
for word in data_name:
    print(word)
    X=fetch_dataset[word]["data"]
    Y=fetch_dataset[word]["target"]
    #print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    minor_table=[]
    major_table=[]
    both_table=[]
    for dim in range(1,30):
        minor_line=[]
        major_line=[]
        both_line=[]
        for split in test_split:
            if d <= dim:
                break
            minor_score=[]
            major_score=[]
            result=[]
            for random_value in range(5):
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))
                
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                LR.fit(Self_train,Self_label)
                value1=LR.predict_log_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=LR.predict_log_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
                Self_train=negative[:]
                if d <= dim:
                    Self_label=np.zeros(len(negative)*(d))
                else:
                    Self_label=np.zeros(len(negative)*(dim+1))
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        order=changelist[n]
                        #print(order)
                        increased=negative[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(negative)*(n+1):len(negative)*(n+2)]=n+1
                LR.fit(Self_train,Self_label)

                value2=LR.predict_log_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value2+=LR.predict_log_proba(Self_test).T[n+1]
                major_score.append(roc_auc_score(y_test,-value2))
                #print(len(LR.predict_log_proba(X_test).T))
                final_score=value1-value2
                result.append(roc_auc_score(y_test,final_score))
            minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            major_line.append(np.array([np.array(major_score).mean(),np.array(major_score).std()]))
            both_line.append(np.array([np.array(result).mean(),np.array(result).std()]))
        minor_table.append(minor_line)
        major_table.append(major_line)
        both_table.append(both_line)
    minor_result.append(minor_table)
    major_result.append(major_table)
    both_result.append(both_table)
makefile(minor_result,"LRmin_5AUC.pkl")
makefile(major_result,"LRmaj_5AUC.pkl")
makefile(both_result,"LRensemble_5AUC.pkl")

In [ ]:
finish=dict()

In [ ]:

test_split=[0.2,0.4,0.9]
minor_result=[]
major_result=[]
both_result=[]
for word in data_name:
    if word in finish:
        pass
    else:
        print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        #print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        original=[]
        for n in range(d):
            original.append(n)
        changelist=[]
        for n in range(1,d):
            change=original[n:d]+original[0:n]
            changelist.append(change)
        minor_table=[]
        major_table=[]
        both_table=[]
        for dim in range(1,30):
            if dim%10==0:
                print(dim)
            minor_line=[]
            major_line=[]
            both_line=[]
            for split in test_split:
                if d <= dim:
                    break
                minor_score=[]
                major_score=[]
                result=[]
                for random_value in range(5):
                    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                    positive=X_train[y_train==1]
                    negative=X_train[y_train==-1]
                    Self_train=positive[:]
                    if d <= dim:
                        Self_label=np.zeros(len(positive)*(d))
                    else:
                        Self_label=np.zeros(len(positive)*(dim+1))

                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            increased=positive[:,order]
                            Self_train=np.concatenate([Self_train,increased])
                            Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                    NB.fit(Self_train,Self_label)
                    value1=NB.predict_log_proba(X_test).T[0]
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            Self_test = X_test[:,order]
                            value1+=NB.predict_log_proba(Self_test).T[n+1]

                    minor_score.append(roc_auc_score(y_test,value1))
                    Self_train=negative[:]
                    if d <= dim:
                        Self_label=np.zeros(len(negative)*(d))
                    else:
                        Self_label=np.zeros(len(negative)*(dim+1))
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            order=changelist[n]
                            #print(order)
                            increased=negative[:,order]
                            Self_train=np.concatenate([Self_train,increased])
                            Self_label[len(negative)*(n+1):len(negative)*(n+2)]=n+1
                    NB.fit(Self_train,Self_label)

                    value2=NB.predict_log_proba(X_test).T[0]
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            Self_test = X_test[:,order]
                            value2+=NB.predict_log_proba(Self_test).T[n+1]
                    major_score.append(roc_auc_score(y_test,-value2))
                    #print(len(LR.predict_log_proba(X_test).T))
                    final_score=value1-value2
                    result.append(roc_auc_score(y_test,final_score))
                minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
                major_line.append(np.array([np.array(major_score).mean(),np.array(major_score).std()]))
                both_line.append(np.array([np.array(result).mean(),np.array(result).std()]))
            minor_table.append(minor_line)
            major_table.append(major_line)
            both_table.append(both_line)
        minor_result.append(minor_table)
        major_result.append(major_table)
        both_result.append(both_table)
        finish[word]=1
makefile(minor_result,"NBmin_5AUC.pkl")
makefile(major_result,"NBmaj_5AUC.pkl")
makefile(both_result,"NBensemble_5AUC.pkl")

In [ ]:
finish=dict()

In [ ]:

test_split=[0.2,0.4,0.6,0.8,0.9]
minor_result=[]
major_result=[]
both_result=[]
for word in data_name:
    if word in finish:
        pass
    else:
        print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        #print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        original=[]
        for n in range(d):
            original.append(n)
        changelist=[]
        for n in range(1,d):
            change=original[n:d]+original[0:n]
            changelist.append(change)
        minor_table=[]
        major_table=[]
        both_table=[]
        for dim in range(1,30):
            if dim%10==0:
                print(dim)
            minor_line=[]
            major_line=[]
            both_line=[]
            for split in test_split:
                if d <= dim:
                    break
                minor_score=[]
                major_score=[]
                result=[]
                for random_value in range(5):
                    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                    positive=X_train[y_train==1]
                    negative=X_train[y_train==-1]
                    Self_train=positive[:]
                    if d <= dim:
                        Self_label=np.zeros(len(positive)*(d))
                    else:
                        Self_label=np.zeros(len(positive)*(dim+1))

                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            increased=positive[:,order]
                            Self_train=np.concatenate([Self_train,increased])
                            Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                    MLP.fit(Self_train,Self_label)
                    value1=MLP.predict_proba(X_test).T[0]
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            Self_test = X_test[:,order]
                            value1+=MLP.predict_proba(Self_test).T[n+1]

                    minor_score.append(roc_auc_score(y_test,value1))
                    Self_train=negative[:]
                    if d <= dim:
                        Self_label=np.zeros(len(negative)*(d))
                    else:
                        Self_label=np.zeros(len(negative)*(dim+1))
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            order=changelist[n]
                            #print(order)
                            increased=negative[:,order]
                            Self_train=np.concatenate([Self_train,increased])
                            Self_label[len(negative)*(n+1):len(negative)*(n+2)]=n+1
                    MLP.fit(Self_train,Self_label)

                    value2=MLP.predict_proba(X_test).T[0]
                    for n in range(dim):
                        if n == d-1:
                            break
                        else:
                            #generate self-labeled data
                            order=changelist[n]
                            Self_test = X_test[:,order]
                            value2+=MLP.predict_proba(Self_test).T[n+1]
                    major_score.append(roc_auc_score(y_test,-value2))
                    #print(len(LR.predict_log_proba(X_test).T))
                    final_score=value1-value2
                    result.append(roc_auc_score(y_test,final_score))
                minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
                major_line.append(np.array([np.array(major_score).mean(),np.array(major_score).std()]))
                both_line.append(np.array([np.array(result).mean(),np.array(result).std()]))
            minor_table.append(minor_line)
            major_table.append(major_line)
            both_table.append(both_line)
        minor_result.append(minor_table)
        major_result.append(major_table)
        both_result.append(both_table)
        finish[word]=1
makefile(minor_result,"MLPmin_5AUC.pkl")
makefile(major_result,"MLPmaj_5AUC.pkl")
makefile(both_result,"MLPensemble_5AUC.pkl")

In [ ]:
from sklearn.datasets import fetch_covtype
file=fetch_covtype()
data=file["data"]
target=file["target"]
labels=dict()
for word in target:
    if word in labels:
        labels[word]+=1
    else:
        labels[word]=1

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=28
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                DT.fit(Self_train,Self_label)
                value1=DT.predict_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=DT.predict_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-DT-COV.pkl")

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    X=data[:]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=11
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                LR.fit(Self_train,Self_label)
                value1=LR.predict_log_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=LR.predict_log_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-LR-COV.pkl")

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=29
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                NB.fit(Self_train,Self_label)
                value1=NB.predict_log_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=NB.predict_log_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-GNB-COV.pkl")

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=7
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                MLP.fit(Self_train,Self_label)
                value1=MLP.predict_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=MLP.predict_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-MLP-COV.pkl")

In [ ]:
from sklearn.datasets import fetch_kddcup99
file=fetch_kddcup99()
data=file["data"]
target=file["target"]
data=np.concatenate([data[:,0:1],data[:,4:]],axis=1).astype("float")
labels=dict()
for word in target:
    if word in labels:
        labels[word]+=1
    else:
        labels[word]=1

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=28
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                DT.fit(Self_train,Self_label)
                value1=DT.predict_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=DT.predict_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-DT-KDD.pkl")

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    X=data[:]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=11
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                LR.fit(Self_train,Self_label)
                value1=LR.predict_log_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=LR.predict_log_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-LR-KDD.pkl")

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=29
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                NB.fit(Self_train,Self_label)
                value1=NB.predict_log_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=NB.predict_log_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-GNB-KDD.pkl")

In [ ]:
minor_result=[]
for word in labels:
    Y=np.zeros(len(target))
    Y[:]=-1
    Y[target==word]=1
    test_split=[0.2,0.4,0.9]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0)+0.0000001)
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    original=[]
    for n in range(d):
        original.append(n)
    changelist=[]
    for n in range(1,d):
        change=original[n:d]+original[0:n]
        changelist.append(change)
    dim=7
    minor_line=[]
    #major_line=[]
    both_line=[]
    for split in test_split:
        minor_score=[]
        
        for random_value in range(5):
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=random_value,stratify=Y)
                positive=X_train[y_train==1]
                #negative=X_train[y_train==-1]
                Self_train=positive[:]
                if d <= dim:
                    Self_label=np.zeros(len(positive)*(d))
                else:
                    Self_label=np.zeros(len(positive)*(dim+1))

                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        increased=positive[:,order]
                        Self_train=np.concatenate([Self_train,increased])
                        Self_label[len(positive)*(n+1):len(positive)*(n+2)]=n+1

                MLP.fit(Self_train,Self_label)
                value1=MLP.predict_proba(X_test).T[0]
                for n in range(dim):
                    if n == d-1:
                        break
                    else:
                        #generate self-labeled data
                        order=changelist[n]
                        Self_test = X_test[:,order]
                        value1+=MLP.predict_proba(Self_test).T[n+1]

                minor_score.append(roc_auc_score(y_test,value1))
            except:
                minor_score.append(0.5)
        print(np.array(minor_score).mean())
        minor_line.append(np.array([np.array(minor_score).mean(),np.array(minor_score).std()]))
            
        
    minor_result.append(minor_line)
    #major_result.append(major_table)
    #both_result.append(both_table)
makefile(minor_result,"OCFSP-MLP-KDD.pkl")